# Generating and adding Data to Firestore

This notebook demonstrates retrieving the Data  from the APIs mentioned in the Bachelor Thesis. It shows the steps made to get the data, scrape the data and uploading the data to Firestore.

It should be mentioned that within the scope of this Jupyter notebook there will be no explanations regarding the chosen data formats and data structures. Everything worth knowing can be found in the bachelor thesis.

## Contents
1. [Getting the Data of the apimedic-api](#getData)

    1.1 [Body Parts](#getBodyParts)
    
    1.2 [Symptoms](#getSymptoms)
    
    1.3 [Diseases](#getDiseases)
    
    
2. [Adding the Data to Firestore](#addData) 

    2.1 [Body Parts](#addBodyParts)
    
    2.2 [Symptoms](#addSymptoms)
    
    2.3 [Diseases](#addDiseases)

In [91]:
# DO THE NECESSARY IMPORTS
import time
import requests
import ssl

## 1. GETTING DATA OF THE MEDIC-API<a id='getData'></a>

The first step in creating the disease detection database is to find an API that will provide the appropriate data. As already mentioned earlier on in the bachelor thesis, this application will be build to work with the data of the medic api: https://apimedic.com. The first step is to retrieve the data of the apimedic-API.

The API can be testet by anyone who created an account and is logged in on this site: https://apimedic.com/apitest

## 1.1 Body Parts<a id='getBodyParts'></a>
First, the different body parts should be pulled from the API, as they form an important attribute of the symptoms.

In [54]:
# BODYPART CLASS
class BodyPart:
    def __init__(bodyPart, pBodyPartID, pBodyPartMedicIP, pName):
        bodyPart.id = pBodyPartID
        # THIS FIELD IS NOT REQUIRED IN THE DATABASE LATER ON, BUT REQUIRED FOR AN API CALL LATER ON
        bodyPart.medicID = pBodyPartMedicIP
        bodyPart.name = pName

In [21]:
# GET BODYPARTS OF APIMEDIC
allBodyPartsURL = "https://healthservice.priaid.ch/body/locations?token=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJlbWFpbCI6ImFucDc1MzFAdGhpLmRlIiwicm9sZSI6IlVzZXIiLCJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9zaWQiOiI4ODM5IiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy92ZXJzaW9uIjoiMTA5IiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9saW1pdCI6IjEwMCIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcCI6IkJhc2ljIiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9sYW5ndWFnZSI6ImVuLWdiIiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy9leHBpcmF0aW9uIjoiMjA5OS0xMi0zMSIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcHN0YXJ0IjoiMjAyMi0xMS0yOSIsImlzcyI6Imh0dHBzOi8vYXV0aHNlcnZpY2UucHJpYWlkLmNoIiwiYXVkIjoiaHR0cHM6Ly9oZWFsdGhzZXJ2aWNlLnByaWFpZC5jaCIsImV4cCI6MTY3MDM1MDQwNywibmJmIjoxNjcwMzQzMjA3fQ.mQLuSaxmG3YeGkWj754t5pIypJuniKtZMBX7xdSichU&format=json&language=en-gb"
allBodyPartsResponse = requests.request("GET", allBodyPartsURL)
allBodyPartsData = allBodyPartsResponse.json()

bodyParts = []
# ITERATE TROUGH THE RECEIVED DATA
for bodyPart in allBodyPartsData:
    # BODY PARTS GOT SUB-BODYPARTS, FOR EXAMPLE: ARMS & SHOULDER -> FINGER, FOREARM & ELBOW, ...
    # NOW EVERY SUB-BODYPART IS GETTING CALLED
    bodyPartURL = "https://healthservice.priaid.ch/body/locations/"+ str(bodyPart["ID"]) +"?token=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJlbWFpbCI6ImFucDc1MzFAdGhpLmRlIiwicm9sZSI6IlVzZXIiLCJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9zaWQiOiI4ODM5IiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy92ZXJzaW9uIjoiMTA5IiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9saW1pdCI6IjEwMCIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcCI6IkJhc2ljIiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9sYW5ndWFnZSI6ImVuLWdiIiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy9leHBpcmF0aW9uIjoiMjA5OS0xMi0zMSIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcHN0YXJ0IjoiMjAyMi0xMS0yOSIsImlzcyI6Imh0dHBzOi8vYXV0aHNlcnZpY2UucHJpYWlkLmNoIiwiYXVkIjoiaHR0cHM6Ly9oZWFsdGhzZXJ2aWNlLnByaWFpZC5jaCIsImV4cCI6MTY3MDM1MDM0NCwibmJmIjoxNjcwMzQzMTQ0fQ.usfKGfIblvToZpm1KhlfamAYTaeKDn1Q2g1WjswJALY&format=json&language=en-gb"
    bodyPartResponse = requests.request("GET", bodyPartURL)
    bodyPartData = bodyPartResponse.json()
    # ITERATE TROUGH SUB-BODYPARTS
    for bodyPart in bodyPartData:
        # BUILD BODYPART-INSTANCES
        # THE SUB-BODYPARTS WILL BE THE ONES STORED IN THE FIRESTORE DATABASE
        bodyPart = BodyPart(bodyPart["Name"].replace(" ", "_").lower(), bodyPart["ID"], bodyPart["Name"])
        bodyParts.append(bodyPart)
    
print("GOT " + str(len(bodyParts)) + " BODY PARTS")

GOT 25 BODY PARTS


Something very important to mention is, that it is surely not the safest option, that the object-ID is based on its name. The reason why it is done here, is solely due to the fact that it is easier to understand screenshots and texts later on in the bachelor thesis. On release of this application, the best idea would to generate some Hash-based IDs for each document.

## 1.1 Symptoms <a id='getSymptoms'></a>

The symptoms can now be created, with help of the body parts. 

In [55]:
# SYMPTOM CLASS
class Symptom:
    def __init__(symptom, pSymptomID, pName, pCauses, pProposedSymptoms, pBodyParts):
        symptom.id = pSymptomID
        symptom.name = pName
        symptom.causes = pCauses
        symptom.proposedSymptoms = pProposedSymptoms
        symptom.bodyParts = pBodyParts

In [53]:
# GET SYMPTOMS OF APIMEDIC
stringURLSymptoms = "https://healthservice.priaid.ch/symptoms?token=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJlbWFpbCI6ImFucDc1MzFAdGhpLmRlIiwicm9sZSI6IlVzZXIiLCJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9zaWQiOiI4ODM5IiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy92ZXJzaW9uIjoiMTA5IiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9saW1pdCI6IjEwMCIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcCI6IkJhc2ljIiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9sYW5ndWFnZSI6ImVuLWdiIiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy9leHBpcmF0aW9uIjoiMjA5OS0xMi0zMSIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcHN0YXJ0IjoiMjAyMi0xMS0yOSIsImlzcyI6Imh0dHBzOi8vYXV0aHNlcnZpY2UucHJpYWlkLmNoIiwiYXVkIjoiaHR0cHM6Ly9oZWFsdGhzZXJ2aWNlLnByaWFpZC5jaCIsImV4cCI6MTY3MDM2NjczNiwibmJmIjoxNjcwMzU5NTM2fQ.v4wFvowgD-pRV9rYnLe7PE6Hm86VuoSH-J9jt4duLkU&format=json&language=en-gb"
responseSymptoms = requests.request("GET", stringURLSymptoms)
dataSymptoms = responseSymptoms.json()

# ITERATE TROUGH DATA AND STORE IN LIST
symptoms = []
for symptom in dataSymptoms:
    stringProposedSymptomsURL = "https://healthservice.priaid.ch/symptoms/proposed?symptoms=[" + str(symptom["ID"]) + "]&gender=female&year_of_birth=2001&token=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJlbWFpbCI6ImFucDc1MzFAdGhpLmRlIiwicm9sZSI6IlVzZXIiLCJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9zaWQiOiI4ODM5IiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy92ZXJzaW9uIjoiMTA5IiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9saW1pdCI6IjEwMCIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcCI6IkJhc2ljIiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9sYW5ndWFnZSI6ImVuLWdiIiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy9leHBpcmF0aW9uIjoiMjA5OS0xMi0zMSIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcHN0YXJ0IjoiMjAyMi0xMS0yOSIsImlzcyI6Imh0dHBzOi8vYXV0aHNlcnZpY2UucHJpYWlkLmNoIiwiYXVkIjoiaHR0cHM6Ly9oZWFsdGhzZXJ2aWNlLnByaWFpZC5jaCIsImV4cCI6MTY3MDM2Njc2NSwibmJmIjoxNjcwMzU5NTY1fQ.pL3pqEcRk438Mzc1RF7GuCT8hQBU-Kp5l7ZaHU614hs&format=json&language=en-gb"
    responseProposedSymptoms = requests.request("GET", stringProposedSymptomsURL)
    dataProposedSymptoms = responseProposedSymptoms.json()
    
    # GET ALL PROPOSEDSYMPTOMS FOR SPECIFIC SYMPTOM AND STORE IN LIST
    proposedSymptoms = []
    for sym in dataProposedSymptoms:
        symID = sym["Name"].replace(" ", "_").lower()
        proposedSymptoms.append(symID)
    
    # GET ALL BODYPARTS THAT COULD BE AFFECTED BY THE SYMPTOM
    bodyPartsSymptom = []
    genders = ["0", "1", "2" ,"3"]
    for bodyPart in bodyParts:
        for gender in genders:
            stringBodyPartsOfSymptom = "https://healthservice.priaid.ch/symptoms/" + str(bodyPart.medicID) + "/" + gender +"?token=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJlbWFpbCI6ImFucDc1MzFAdGhpLmRlIiwicm9sZSI6IlVzZXIiLCJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9zaWQiOiI4ODM5IiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy92ZXJzaW9uIjoiMTA5IiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9saW1pdCI6IjEwMCIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcCI6IkJhc2ljIiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9sYW5ndWFnZSI6ImVuLWdiIiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy9leHBpcmF0aW9uIjoiMjA5OS0xMi0zMSIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcHN0YXJ0IjoiMjAyMi0xMS0yOSIsImlzcyI6Imh0dHBzOi8vYXV0aHNlcnZpY2UucHJpYWlkLmNoIiwiYXVkIjoiaHR0cHM6Ly9oZWFsdGhzZXJ2aWNlLnByaWFpZC5jaCIsImV4cCI6MTY3MDM2Njc2NSwibmJmIjoxNjcwMzU5NTY1fQ.pL3pqEcRk438Mzc1RF7GuCT8hQBU-Kp5l7ZaHU614hs&format=json&language=en-gb"
            responseBodyPartsOfSymptom = requests.request("GET", stringBodyPartsOfSymptom)
            dataBodyPartsOfSymptom = responseBodyPartsOfSymptom.json()
            for part in dataBodyPartsOfSymptom:
                if symptom["ID"] in part["HealthSymptomLocationIDs"]:
                    bodyPartsSymptom.append(bodyPart.id)
    
    bodyPartsSymptom = list(dict.fromkeys(bodyPartsSymptom))
    # CREATE SYMPTOM AND APPEND TO SYMPTOMLIST
    # SADLY, APIMEDIC DOES NOT HAVE CAUSES FOR THEIR SYMPTOMS AND DISEASES -> [""]
    symptom = Symptom(symptom["Name"].replace(" ", "_").lower(), symptom["Name"], [""], proposedSymptoms, bodyPartsSymptom)
    symptoms.append(symptom)
    print(symptom.name, symptom.proposedSymptoms, symptom.bodyParts)

# PRINT THE RESULTS, ALL OF THOSE CAN BE ADDED TO FIRESTORE
print("GOT " + str(len(symptoms)) + " SYMPTOMS")

Abdominal guarding ['abdominal_pain', 'rebound_tenderness', 'lower_abdominal_pain', 'diarrhea', 'vomiting', 'nausea'] []
Abdominal pain ['abdominal_pain_associated_with_menstruation', 'diarrhea', 'missed_period', 'heartburn', 'decreased_urine_stream', 'increased_appetite'] ['buttocks_&_rectum', 'genitals_&_groin', 'hips_&_hip_joint', 'arms_general', 'finger', 'hand_&_wrist', 'back', 'forehead_&_head_in_general', 'nose,_ears,_throat_&_neck', 'foot', 'legs_general', 'lower_leg_&_ankle', 'thigh_&_knee', 'toes', 'general,_joints_&_other', 'skin']
Abdominal pain associated with menstruation ['abdominal_pain', 'back_pain', 'tiredness', 'lower_abdominal_pain', 'bleeding_from_vagina', 'dizziness'] []
Absence of a pulse ['diarrhea', 'memory_gap', 'sputum', 'urination_during_the_night', 'cold_feet', 'increased_thirst'] []
Aggressiveness ['reduced_appetite', 'mood_swings', 'increased_appetite', 'tiredness', 'sleeplessness', 'difficulty_to_concentrate'] []
Agitation ['neck_stiffness', 'cough', 'in

Difficulty in finding words ['headache', 'problems_with_the_sense_of_touch_in_the_face', 'increased_thirst', 'immobilization', 'increased_urine_quantity', 'difficulty_in_speaking'] []
Difficulty in speaking ['memory_gap', 'diarrhea', 'immobilization', 'difficulty_to_concentrate', 'shortness_of_breath', 'limited_mobility_of_the_leg'] []
Difficulty in swallowing ['mouth_ulcers', 'earache', 'increased_appetite', 'difficulty_in_speaking', 'immobilization', 'limited_mobility_of_the_leg'] []
Difficulty to concentrate ['headache', 'tiredness', 'sleeplessness', 'increased_appetite', 'reduced_appetite', 'mood_swings'] []
Difficulty to learn ['headache', 'memory_gap', 'nausea', 'double_vision,_acute-onset', 'double_vision', 'drowsiness'] []
Difficulty with gait ['overweight', 'flatulence', 'hip_pain', 'limited_mobility_of_the_leg', 'limited_mobility_of_the_back', 'sweating'] []
Discoloration of nails ['changes_in_the_nails', 'skin_rash', 'skin_redness', 'itching_of_skin', 'skin_lesion', 'dry_ski

Headache ['burning_nose', 'runny_nose', 'missed_period', 'stuffy_nose', 'menstruation_disorder', 'sore_throat'] []
Hearing loss ['ringing_in_the_ear', 'impaired_hearing', 'feeling_of_pressure_in_the_ear', 'impaired_balance'] []
Heart murmur ['urination_during_the_night', 'cough', 'shortness_of_breath', 'changes_in_the_nails', 'ankle_swelling', 'night_cough'] []
Heartburn ['cough', 'pain_on_swallowing', 'vomiting_blood', 'diarrhea', 'missed_period', 'increased_appetite'] ['arms_general', 'finger', 'hand_&_wrist', 'foot', 'legs_general', 'toes', 'general,_joints_&_other']
Hiccups ['heartburn', 'cough', 'vomiting_blood', 'increased_appetite', 'diarrhea', 'mouth_ulcers'] []
Hip deformity ['bone_fracture', 'hip_pain', 'immobilization', 'limited_mobility_of_the_leg', 'joint_pain', 'joint_instability'] []
Hip pain ['overweight', 'stress-related_leg_pain', 'limited_mobility_of_the_leg', 'bone_fracture', 'neck_pain', 'joint_pain'] []
Hoarseness ['sore_throat', 'heartburn', 'reduced_appetite', '

Missed period ['menstruation_disorder', 'vomiting', 'nausea', 'reduced_appetite', 'joylessness', 'tiredness'] []
Moist and softened skin ['reduced_appetite', 'menstruation_disorder', 'missed_period', 'joylessness', 'tiredness', 'dry_skin'] []
Mood swings ['missed_period', 'reduced_appetite', 'increased_appetite', 'diarrhea', 'menstruation_disorder', 'tiredness'] []
Morning stiffness ['neck_stiffness', 'hip_pain', 'limited_mobility_of_the_back', 'limited_mobility_of_the_leg', 'neck_pain', 'stress-related_leg_pain'] []
Mouth pain ['mouth_ulcers', 'sore_throat', 'heartburn', 'cough', 'earache', 'lip_swelling'] []
Mouth ulcers ['mouth_pain', 'toothache', 'dry_mouth', 'increased_appetite', 'diarrhea', 'heartburn'] []
Muscle pain ['limited_mobility_of_the_leg', 'sore_throat', 'reduced_appetite', 'missed_period', 'menstruation_disorder', 'joylessness'] []
Muscle stiffness ['increased_appetite', 'diarrhea', 'mouth_ulcers', 'heartburn', 'skin_wheal', 'abdominal_pain'] []
Muscle weakness ['stuff

Side pain ['decreased_urine_stream', 'less_than_3_defecations_per_week', 'flatulence', 'abdominal_pain', 'hard_defecation', 'urge_to_urinate'] []
Skin lesion ['blackhead', 'skin_nodules', 'mouth_ulcers', 'diarrhea', 'increased_appetite', 'heartburn'] ['arms_general', 'forearm_&_elbow', 'upper_arm_&_shoulder']
Skin nodules ['blackhead', 'skin_lesion', 'wound', 'itching_of_skin', 'skin_rash', 'hardening_of_the_skin'] ['abdomen', 'finger', 'chest', 'face_&_eyes', 'forehead_&_head_in_general', 'mouth_&_jaw', 'nose,_ears,_throat_&_neck', 'general,_joints_&_other', 'skin']
Skin rash ['coarsening_of_the_skin_structure', 'neck_stiffness', 'skin_wheal', 'diarrhea', 'cough', 'mouth_ulcers'] []
Skin redness ['skin_wheal', 'skin_rash', 'increased_appetite', 'diarrhea', 'chills', 'mouth_ulcers'] []
Skin thickening ['coarsening_of_the_skin_structure', 'hardening_of_the_skin', 'dry_skin', 'itching_of_skin', 'skin_rash', 'skin_nodules'] []
Skin wheal ['itching_of_skin', 'skin_rash', 'lip_swelling', 'i

### 1.2 Diseases<a id='getDiseases'></a>
Now, after retrieving the symptoms of the API, the issues (diseases) need to be retrieved.

In [87]:
# DISEASE CLASS FOR FIRESTORE
class Disease:
    def __init__(disease, pDiseaseID, pName, pDescription, pSymptoms, pCauses, pTreatment):
        disease.id = pDiseaseID
        disease.name = pName
        disease.description = pDescription
        disease.symptoms = pSymptoms
        disease.causes = pCauses
        disease.treatment = pTreatment

    # FIND THE SYMPTOMS IN THE STRING RETRIEVED FROM NHS DATABASE
    def editSymptoms(self):
        print(self.symptoms)
        symps = self.symptoms.split(",")
        print(symps)
        changer = []
        for x in symps:
            print(x)
            x = x.replace(" ", "_").lower()
            changer.append(x)
        self.symptoms = changer

In [88]:
# GET ISSUES OF APIMEDIC
urlIssues = "https://healthservice.priaid.ch/issues?token=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJlbWFpbCI6ImFucDc1MzFAdGhpLmRlIiwicm9sZSI6IlVzZXIiLCJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9zaWQiOiI4ODM5IiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy92ZXJzaW9uIjoiMTA5IiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9saW1pdCI6IjEwMCIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcCI6IkJhc2ljIiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9sYW5ndWFnZSI6ImVuLWdiIiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy9leHBpcmF0aW9uIjoiMjA5OS0xMi0zMSIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcHN0YXJ0IjoiMjAyMi0xMS0yOSIsImlzcyI6Imh0dHBzOi8vYXV0aHNlcnZpY2UucHJpYWlkLmNoIiwiYXVkIjoiaHR0cHM6Ly9oZWFsdGhzZXJ2aWNlLnByaWFpZC5jaCIsImV4cCI6MTY3MDM3OTAxMSwibmJmIjoxNjcwMzcxODExfQ.q1AYVNkWiPWRmvaAXuFIzu2lf7mbfUQYKEfFe6gDQWg&format=json&language=en-gb"
responseIssues = requests.request("GET", urlIssues)
dataIssues = responseIssues.json()

issues = []
for piece in dataIssues:
    issueURL = "https://healthservice.priaid.ch/issues/"+ str(piece["ID"]) +"/info?token=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJlbWFpbCI6ImFucDc1MzFAdGhpLmRlIiwicm9sZSI6IlVzZXIiLCJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9zaWQiOiI4ODM5IiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy92ZXJzaW9uIjoiMTA5IiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9saW1pdCI6IjEwMCIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcCI6IkJhc2ljIiwiaHR0cDovL2V4YW1wbGUub3JnL2NsYWltcy9sYW5ndWFnZSI6ImVuLWdiIiwiaHR0cDovL3NjaGVtYXMubWljcm9zb2Z0LmNvbS93cy8yMDA4LzA2L2lkZW50aXR5L2NsYWltcy9leHBpcmF0aW9uIjoiMjA5OS0xMi0zMSIsImh0dHA6Ly9leGFtcGxlLm9yZy9jbGFpbXMvbWVtYmVyc2hpcHN0YXJ0IjoiMjAyMi0xMS0yOSIsImlzcyI6Imh0dHBzOi8vYXV0aHNlcnZpY2UucHJpYWlkLmNoIiwiYXVkIjoiaHR0cHM6Ly9oZWFsdGhzZXJ2aWNlLnByaWFpZC5jaCIsImV4cCI6MTY3MDM3OTA0MCwibmJmIjoxNjcwMzcxODQwfQ.gfmm5Y6kOBDCkl86tR03RcpTLAsB4QtTUFYqc-Xsnbw&format=json&language=en-gb"
    issueResponse = requests.request("GET", issueURL)
    data = issueResponse.json()
    issue = Disease(data["Name"].replace(" ", "_").lower(), data["Name"], data["Description"], data["PossibleSymptoms"], [""] , data["TreatmentDescription"])
    issue.editSymptoms()
    issues.append(issue)
    print(issue.name, issue.description, issue.symptoms)

    
print("GOT " + str(len(issues)) + " ISSUES")

Bulging abdominal wall,Nausea,Abdominal pain,Swelling in the genital area,Swelling of the testicles,Testicular pain,Distended abdomen
['Bulging abdominal wall', 'Nausea', 'Abdominal pain', 'Swelling in the genital area', 'Swelling of the testicles', 'Testicular pain', 'Distended abdomen']
Bulging abdominal wall
Nausea
Abdominal pain
Swelling in the genital area
Swelling of the testicles
Testicular pain
Distended abdomen
Abdominal hernia There are different causes and localisations for abdominal wall hernias. The weakness in the abdomen can be congenital or acquired, and depending on their location, can be seen or felt when the intestinal loops emerge. The intestinal loops may emerge permanently or slide back. ['bulging_abdominal_wall', 'nausea', 'abdominal_pain', 'swelling_in_the_genital_area', 'swelling_of_the_testicles', 'testicular_pain', 'distended_abdomen']
Abdominal pain,Bleeding from vagina,Back pain
['Abdominal pain', 'Bleeding from vagina', 'Back pain']
Abdominal pain
Bleeding

Bone fracture,Immobilization,Arm swelling,Joint swelling,Joint pain,Hand swelling,Blue spot on skin,Pain in the bones,Arm pain,Joint instability
['Bone fracture', 'Immobilization', 'Arm swelling', 'Joint swelling', 'Joint pain', 'Hand swelling', 'Blue spot on skin', 'Pain in the bones', 'Arm pain', 'Joint instability']
Bone fracture
Immobilization
Arm swelling
Joint swelling
Joint pain
Hand swelling
Blue spot on skin
Pain in the bones
Arm pain
Joint instability
Accident-related bone fracture of arm Accident-related bone fractures can be a disruption in any part of human bones. The disruption results from accidents. The form of the fracture also depends on how the bones are broken. ['bone_fracture', 'immobilization', 'arm_swelling', 'joint_swelling', 'joint_pain', 'hand_swelling', 'blue_spot_on_skin', 'pain_in_the_bones', 'arm_pain', 'joint_instability']
Arm pain,Arm swelling,Blue spot on skin,Bone fracture,Elbow pain,Immobilization,Joint instability,Joint pain,Joint swelling,Pain in th

Blue spot on skin,Bone fracture,Foot pain,Toe deformity,Toe swelling,Immobilization,Pain in the bones,Joint pain
['Blue spot on skin', 'Bone fracture', 'Foot pain', 'Toe deformity', 'Toe swelling', 'Immobilization', 'Pain in the bones', 'Joint pain']
Blue spot on skin
Bone fracture
Foot pain
Toe deformity
Toe swelling
Immobilization
Pain in the bones
Joint pain
Accident-related bone fracture of toe Accident-related bone fractures can be a disruption in any part of human bones. The disruption results from accidents. The form of the fracture also depends on how the bones are broken. ['blue_spot_on_skin', 'bone_fracture', 'foot_pain', 'toe_deformity', 'toe_swelling', 'immobilization', 'pain_in_the_bones', 'joint_pain']
Blue spot on skin,Bone fracture,Hand pain,Hand swelling,Immobilization,Joint instability,Joint pain,Joint swelling,Pain in the bones
['Blue spot on skin', 'Bone fracture', 'Hand pain', 'Hand swelling', 'Immobilization', 'Joint instability', 'Joint pain', 'Joint swelling', '

Cheek swelling,Dizziness,Nausea,Shortness of breath,Skin rash,Unconsciousness, short,Arm swelling,Leg swelling,Lip swelling,Facial swelling,Foot swelling,Toe swelling,Tongue swelling,Cold sweats,Anxiety,Diarrhea,Eyelid swelling,Wheezing,Feeling faint,Blackening of vision
['Cheek swelling', 'Dizziness', 'Nausea', 'Shortness of breath', 'Skin rash', 'Unconsciousness', ' short', 'Arm swelling', 'Leg swelling', 'Lip swelling', 'Facial swelling', 'Foot swelling', 'Toe swelling', 'Tongue swelling', 'Cold sweats', 'Anxiety', 'Diarrhea', 'Eyelid swelling', 'Wheezing', 'Feeling faint', 'Blackening of vision']
Cheek swelling
Dizziness
Nausea
Shortness of breath
Skin rash
Unconsciousness
 short
Arm swelling
Leg swelling
Lip swelling
Facial swelling
Foot swelling
Toe swelling
Tongue swelling
Cold sweats
Anxiety
Diarrhea
Eyelid swelling
Wheezing
Feeling faint
Blackening of vision
Allergy to bee or wasp venom Allergy to bee or wasp venom can be uncomfortable but most people recover fast and without 

Drowsiness,Vomiting,Impaired hearing,Sensitivity to noise,Ringing in the ear,Dizziness,Nausea,Feeling of pressure in the ear,Earache,Impaired balance
['Drowsiness', 'Vomiting', 'Impaired hearing', 'Sensitivity to noise', 'Ringing in the ear', 'Dizziness', 'Nausea', 'Feeling of pressure in the ear', 'Earache', 'Impaired balance']
Drowsiness
Vomiting
Impaired hearing
Sensitivity to noise
Ringing in the ear
Dizziness
Nausea
Feeling of pressure in the ear
Earache
Impaired balance
Auditory vertigo Ménière's disease is a disorder of inner ear disorder that can affect hearing and balance, and then it can result in symptoms like tinnitus, hearing loss, dizziness or a feeling of fullness or congestion in the inner ear. It normally affects only one ear. ['drowsiness', 'vomiting', 'impaired_hearing', 'sensitivity_to_noise', 'ringing_in_the_ear', 'dizziness', 'nausea', 'feeling_of_pressure_in_the_ear', 'earache', 'impaired_balance']
Fever,Pain in the limbs,Dry mouth,Joint pain,Chest tightness,Dry 

...

Numbness in the leg,Muscular atrophy in the leg,Muscular weakness in the leg,Foot pain,Tingling
['Numbness in the leg', 'Muscular atrophy in the leg', 'Muscular weakness in the leg', 'Foot pain', 'Tingling']
Numbness in the leg
Muscular atrophy in the leg
Muscular weakness in the leg
Foot pain
Tingling
Tarsal tunnel syndrome A tarsal tunnel syndrome results from an improper compression of the nerve of the tibia (nervus tibialis). Depending on the compression, it may lead to symptoms such as tingling, burning, numbness, or pain along the nerve of the tibia at the inner side and the sole of the affected foot. ['numbness_in_the_leg', 'muscular_atrophy_in_the_leg', 'muscular_weakness_in_the_leg', 'foot_pain', 'tingling']
Blood in stool,Hard defecation,Painful defecation,Cramps,Pain of the anus,Itching of the anus,Difficult defecation,Less than 3 defecations per week
['Blood in stool', 'Hard defecation', 'Painful defecation', 'Cramps', 'Pain of the anus', 'Itching of the anus', 'Difficult d

Face pain,Lip swelling,Mouth pain,Earache,Cheek swelling,Facial swelling,Headache,Swollen glands in the neck,Night sweats,Toothache,Pain when chewing
['Face pain', 'Lip swelling', 'Mouth pain', 'Earache', 'Cheek swelling', 'Facial swelling', 'Headache', 'Swollen glands in the neck', 'Night sweats', 'Toothache', 'Pain when chewing']
Face pain
Lip swelling
Mouth pain
Earache
Cheek swelling
Facial swelling
Headache
Swollen glands in the neck
Night sweats
Toothache
Pain when chewing
Tooth abscess A tooth abscess often results from a bacterial infection in the tooth. Due to the infection, pus can form in the dental pulp. Also teeth decay and periodontal diseases can favor an infection which can lead to a tooth abscess. ['face_pain', 'lip_swelling', 'mouth_pain', 'earache', 'cheek_swelling', 'facial_swelling', 'headache', 'swollen_glands_in_the_neck', 'night_sweats', 'toothache', 'pain_when_chewing']
Swelling in the genital area,Testicular pain,Swelling of the testicles,Skin redness,Nausea,V

Eyelid swelling
['Eyelid swelling']
Eyelid swelling
Ulcer of the skin of the eyelid A chalazion looks similar to a sty. The chalazion is caused by an obstruction of the meibomian gland. The gland is located in the eyelid and produces oily fluids to moisturize the eyes. If this meibomian gland is obstructed it will lead to an accumulation of secretion and thereby lead to a chalazion. ['eyelid_swelling']
Numbness of the hands,Tingling,Finger deformity,Pain radiating to the arm,Arm pain,Numbness in the arm
['Numbness of the hands', 'Tingling', 'Finger deformity', 'Pain radiating to the arm', 'Arm pain', 'Numbness in the arm']
Numbness of the hands
Tingling
Finger deformity
Pain radiating to the arm
Arm pain
Numbness in the arm
Ulnar nerve entrapment Ulnar nerve entrapment, or cubital tunnel syndrome, is a condition that involves stretching or pressure to the ulnar nerve. The common symptoms of cubital tunnel syndrome include numbness or unresponsiveness and tingling in the little finger a

Fever,Joint pain,Weight loss,Pain in the limbs,Skin rash,Cough,Ankle swelling,Headache,Paralysis,Swollen glands in the neck,Swollen glands in the armpit,Swollen glands in the groin,Tiredness,Runny nose,Non-healing skin wound,Vision impairment,Cold hands,Cold feet,Nosebleed,Night sweats,Sore throat,Feeling ill
['Fever', 'Joint pain', 'Weight loss', 'Pain in the limbs', 'Skin rash', 'Cough', 'Ankle swelling', 'Headache', 'Paralysis', 'Swollen glands in the neck', 'Swollen glands in the armpit', 'Swollen glands in the groin', 'Tiredness', 'Runny nose', 'Non-healing skin wound', 'Vision impairment', 'Cold hands', 'Cold feet', 'Nosebleed', 'Night sweats', 'Sore throat', 'Feeling ill']
Fever
Joint pain
Weight loss
Pain in the limbs
Skin rash
Cough
Ankle swelling
Headache
Paralysis
Swollen glands in the neck
Swollen glands in the armpit
Swollen glands in the groin
Tiredness
Runny nose
Non-healing skin wound
Vision impairment
Cold hands
Cold feet
Nosebleed
Night sweats
Sore throat
Feeling ill


Difficulty in swallowing,Mouth pain,Dry mouth
['Difficulty in swallowing', 'Mouth pain', 'Dry mouth']
Difficulty in swallowing
Mouth pain
Dry mouth
Yeast infection Candida is a fungus that lives almost everywhere, including the human body. Usually, a person's immune system keeps it under control. However if a person has an immune system deficiency or he is taking antibiotics, the fungus may multiply and cause a local or even systemic infection called candidiasis. ['difficulty_in_swallowing', 'mouth_pain', 'dry_mouth']
Fever,Headache,Nausea,Black stools,Vomiting blood,Bloody cough,Vomiting,Eye redness,Muscle pain,Blood in stool,Unconsciousness, short
['Fever', 'Headache', 'Nausea', 'Black stools', 'Vomiting blood', 'Bloody cough', 'Vomiting', 'Eye redness', 'Muscle pain', 'Blood in stool', 'Unconsciousness', ' short']
Fever
Headache
Nausea
Black stools
Vomiting blood
Bloody cough
Vomiting
Eye redness
Muscle pain
Blood in stool
Unconsciousness
 short
Yellow plague The yellow fever virus 

## 2. Adding the Data to Firestore  <a id='addData'></a>

In [25]:
pip install --upgrade firebase-admin

     -------------------------------------- 119.4/119.4 kB 7.3 MB/s eta 0:00:00
     -------------------------------------- 105.7/105.7 kB 6.0 MB/s eta 0:00:00
     --------------------------------------- 10.6/10.6 MB 21.1 MB/s eta 0:00:00
     ------------------------------------- 253.3/253.3 kB 15.2 MB/s eta 0:00:00
     -------------------------------------- 120.3/120.3 kB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 61.9/61.9 kB ? eta 0:00:00
     ------------------------------------- 177.0/177.0 kB 11.1 MB/s eta 0:00:00
     ------------------------------------- 218.0/218.0 kB 13.0 MB/s eta 0:00:00
     ------------------------------------- 525.1/525.1 kB 32.2 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 26.5 MB/s eta 0:00:00
     ---------------------------------------- 96.8/96.8 kB ? eta 0:00:00
     ---------------------------------------- 47.9/47.9 kB ? eta 0:00:00
     ---------------------------------------- 77.4/77.4 kB 4.2

In [39]:
import os    
credential_path = "C:\\Users\\Thomazzzz\\Documents\\serviceAccountKey.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials

if not firebase_admin._apps:
    cred = credentials.Certificate("C:\\Users\\Thomazzzz\\Documents\\serviceAccountKey.json") 
    default_app = firebase_admin.initialize_app(cred)


db = firestore.client()


## 2.1 Adding the BodyParts <a id='addBodyParts'></a>

First of all, some more data should be added to the symptomlist (generated in step 1). This allows it, to manually add data to the database. The now added data is based on some research in the NHS-API, since surely some symptoms of the apimedic are not exactly named as they are in the NHS-API. This ensures, that more symptoms are getting detected in the following steps

The same effect can be achievied with adding some more diseases to the issuelist retrieved from the apimedic-api.

In [45]:
try:
    collection = db.collection('body_parts')  # create collection
    for bodyPart in bodyParts:
        res = collection.document(bodyPart.id).set({ # insert document
            'name': bodyPart.name,
            'medic_API_id': bodyPart.medicID,
        })
        print(res)
except:
  print("An exception occurred") 

update_time {
  seconds: 1670359049
  nanos: 738723000
}

update_time {
  seconds: 1670359049
  nanos: 824060000
}

update_time {
  seconds: 1670359049
  nanos: 905168000
}

update_time {
  seconds: 1670359049
  nanos: 994057000
}

update_time {
  seconds: 1670359050
  nanos: 439860000
}

update_time {
  seconds: 1670359050
  nanos: 522530000
}

update_time {
  seconds: 1670359050
  nanos: 606263000
}

update_time {
  seconds: 1670359050
  nanos: 683781000
}

update_time {
  seconds: 1670359050
  nanos: 762217000
}

update_time {
  seconds: 1670359050
  nanos: 837446000
}

update_time {
  seconds: 1670359050
  nanos: 916592000
}

update_time {
  seconds: 1670359050
  nanos: 997159000
}

update_time {
  seconds: 1670359051
  nanos: 83849000
}

update_time {
  seconds: 1670359051
  nanos: 163074000
}

update_time {
  seconds: 1670359051
  nanos: 247767000
}

update_time {
  seconds: 1670359051
  nanos: 307406000
}

update_time {
  seconds: 1670359051
  nanos: 393705000
}

update_time {
 

## 2.2 Adding the Symptoms <a id='addSymptoms'></a>

In [58]:
try:
    collection = db.collection('symptoms')  # create collection
    for symptom in symptoms:
        res = collection.document(symptom.id).set({ # insert document
            'name': symptom.name,
            'causes': symptom.causes,
            'proposed_symptoms': symptom.proposedSymptoms,
            'body_parts' : symptom.bodyParts
        })
        print(res)
except:
  print("An exception occurred") 

update_time {
  seconds: 1670367354
  nanos: 293638000
}

update_time {
  seconds: 1670367354
  nanos: 414523000
}

update_time {
  seconds: 1670367354
  nanos: 513699000
}

update_time {
  seconds: 1670367354
  nanos: 612297000
}

update_time {
  seconds: 1670367354
  nanos: 717346000
}

update_time {
  seconds: 1670367354
  nanos: 844805000
}

update_time {
  seconds: 1670367355
  nanos: 49751000
}

update_time {
  seconds: 1670367355
  nanos: 133301000
}

update_time {
  seconds: 1670367355
  nanos: 306909000
}

update_time {
  seconds: 1670367355
  nanos: 432483000
}

update_time {
  seconds: 1670367355
  nanos: 529242000
}

update_time {
  seconds: 1670367355
  nanos: 637385000
}

update_time {
  seconds: 1670367355
  nanos: 734022000
}

update_time {
  seconds: 1670367355
  nanos: 832929000
}

update_time {
  seconds: 1670367355
  nanos: 937273000
}

update_time {
  seconds: 1670367356
  nanos: 40896000
}

update_time {
  seconds: 1670367356
  nanos: 138199000
}

update_time {
  

update_time {
  seconds: 1670367365
  nanos: 780734000
}

update_time {
  seconds: 1670367365
  nanos: 842532000
}

update_time {
  seconds: 1670367365
  nanos: 903826000
}

update_time {
  seconds: 1670367365
  nanos: 959862000
}

update_time {
  seconds: 1670367366
  nanos: 17699000
}

update_time {
  seconds: 1670367366
  nanos: 107047000
}

update_time {
  seconds: 1670367366
  nanos: 165701000
}

update_time {
  seconds: 1670367366
  nanos: 227148000
}

update_time {
  seconds: 1670367366
  nanos: 284563000
}

update_time {
  seconds: 1670367366
  nanos: 367140000
}

update_time {
  seconds: 1670367366
  nanos: 428813000
}

update_time {
  seconds: 1670367366
  nanos: 484212000
}

update_time {
  seconds: 1670367366
  nanos: 542037000
}

update_time {
  seconds: 1670367366
  nanos: 605238000
}

update_time {
  seconds: 1670367366
  nanos: 662872000
}

update_time {
  seconds: 1670367366
  nanos: 726785000
}

update_time {
  seconds: 1670367366
  nanos: 785061000
}

update_time {
 

## 2.3 Adding the Diseases <a id='addDiseases'></a>

In [89]:
try:
    collection = db.collection('diseases')  # create collection
    for disease in issues:
        res = collection.document(disease.id).set({ # insert document
            'name': disease.name,
            'description': disease.description,
            'symptoms': disease.symptoms,
            'causes' : disease.causes,
            'treatment' : disease.treatment
        })
        print(res)
except Exception as e:
  print(e) 

update_time {
  seconds: 1670373072
  nanos: 700322000
}

update_time {
  seconds: 1670373072
  nanos: 794017000
}

update_time {
  seconds: 1670373072
  nanos: 882295000
}

update_time {
  seconds: 1670373072
  nanos: 971928000
}

update_time {
  seconds: 1670373073
  nanos: 65086000
}

update_time {
  seconds: 1670373073
  nanos: 153727000
}

update_time {
  seconds: 1670373073
  nanos: 282766000
}

update_time {
  seconds: 1670373073
  nanos: 363955000
}

update_time {
  seconds: 1670373073
  nanos: 443691000
}

update_time {
  seconds: 1670373073
  nanos: 532361000
}

update_time {
  seconds: 1670373073
  nanos: 605314000
}

update_time {
  seconds: 1670373073
  nanos: 686128000
}

update_time {
  seconds: 1670373073
  nanos: 768589000
}

update_time {
  seconds: 1670373073
  nanos: 852076000
}

update_time {
  seconds: 1670373073
  nanos: 931036000
}

update_time {
  seconds: 1670373074
  nanos: 17565000
}

update_time {
  seconds: 1670373074
  nanos: 87540000
}

update_time {
  s

update_time {
  seconds: 1670373082
  nanos: 498203000
}

update_time {
  seconds: 1670373082
  nanos: 565487000
}

update_time {
  seconds: 1670373082
  nanos: 623777000
}

update_time {
  seconds: 1670373082
  nanos: 683305000
}

update_time {
  seconds: 1670373082
  nanos: 758699000
}

update_time {
  seconds: 1670373082
  nanos: 816484000
}

update_time {
  seconds: 1670373082
  nanos: 872624000
}

update_time {
  seconds: 1670373082
  nanos: 933130000
}

update_time {
  seconds: 1670373082
  nanos: 990736000
}

update_time {
  seconds: 1670373083
  nanos: 59300000
}

update_time {
  seconds: 1670373083
  nanos: 120055000
}

update_time {
  seconds: 1670373083
  nanos: 178469000
}

update_time {
  seconds: 1670373083
  nanos: 178469000
}

update_time {
  seconds: 1670373083
  nanos: 309004000
}

update_time {
  seconds: 1670373083
  nanos: 370232000
}

update_time {
  seconds: 1670373083
  nanos: 427574000
}

update_time {
  seconds: 1670373083
  nanos: 486176000
}

update_time {
 

update_time {
  seconds: 1670373091
  nanos: 302299000
}

update_time {
  seconds: 1670373091
  nanos: 364024000
}

update_time {
  seconds: 1670373091
  nanos: 422508000
}

update_time {
  seconds: 1670373091
  nanos: 484483000
}

update_time {
  seconds: 1670373091
  nanos: 544344000
}

update_time {
  seconds: 1670373091
  nanos: 603281000
}

update_time {
  seconds: 1670373091
  nanos: 659062000
}

update_time {
  seconds: 1670373091
  nanos: 717745000
}

update_time {
  seconds: 1670373091
  nanos: 775715000
}

update_time {
  seconds: 1670373091
  nanos: 836757000
}

update_time {
  seconds: 1670373091
  nanos: 895044000
}

update_time {
  seconds: 1670373091
  nanos: 955785000
}

update_time {
  seconds: 1670373092
  nanos: 16277000
}

update_time {
  seconds: 1670373092
  nanos: 76454000
}

update_time {
  seconds: 1670373092
  nanos: 134763000
}

update_time {
  seconds: 1670373092
  nanos: 193157000
}

update_time {
  seconds: 1670373092
  nanos: 261904000
}

update_time {
  

update_time {
  seconds: 1670373101
  nanos: 438677000
}

update_time {
  seconds: 1670372828
  nanos: 439539000
}

update_time {
  seconds: 1670373101
  nanos: 565845000
}

update_time {
  seconds: 1670373101
  nanos: 657395000
}

update_time {
  seconds: 1670373101
  nanos: 720190000
}

update_time {
  seconds: 1670373101
  nanos: 797900000
}

update_time {
  seconds: 1670373101
  nanos: 797900000
}

update_time {
  seconds: 1670373101
  nanos: 952248000
}

update_time {
  seconds: 1670373102
  nanos: 8456000
}

update_time {
  seconds: 1670373102
  nanos: 65951000
}

update_time {
  seconds: 1670372828
  nanos: 944114000
}

update_time {
  seconds: 1670372829
  nanos: 355000
}

update_time {
  seconds: 1670373102
  nanos: 263155000
}

update_time {
  seconds: 1670373102
  nanos: 325864000
}

update_time {
  seconds: 1670373102
  nanos: 384772000
}

update_time {
  seconds: 1670373102
  nanos: 464132000
}

update_time {
  seconds: 1670373102
  nanos: 528081000
}

update_time {
  seco

Since the data in the apimedic-API can always change, it is good to delete duplicated data, just in case the lists already contained some of the data that was manually added.